In [1]:
import time
import paho.mqtt.client
import paho.mqtt.publish
import numpy as np
import random
import datetime
import json
import ssl
import sys
import psycopg2
import pandas as pd
from sqlalchemy import create_engine

# Conexión al broker

In [2]:
host = "broker.hivemq.com"

client = paho.mqtt.client.Client("Publicador de estantes", False)

client.qos = 2

client.connect(host=host)

0

# Conexión a la bd

In [3]:
myConnection = psycopg2.connect(
    host='lallah.db.elephantsql.com',
    user ='okgedrkq',
    password='OWJG1VssXGsfuUWTguGQoCmtZ5c5JZIp',
    dbname='okgedrkq'
)

In [4]:
def select(query):
    cur = myConnection.cursor()
    try:
        cur.execute(query)
    except Exception as e:
        myConnection.commit()
        print('Error en el query:', e)
    else:
        records = cur.fetchall()
        cur.close()
        return records

# Usaremos POO para manejar los estantes

In [5]:
class Estante:
    producto = 0
    def __init__(self, id, id_sucursal, capacidad, disponibilidad):
        self.id = id
        self.id_sucursal = id_sucursal
        self.capacidad = capacidad
        self.disponibilidad = disponibilidad

# Traer información de la  BD

In [6]:
# Sucursal 1
query = """ select * from Sucursal
            Where id = 1
"""
sucursal_1 = select(query)
sucursal_1 = sucursal_1[0]

# Sucursal 2
query = """ select * from Sucursal
            Where id = 2
"""
sucursal_2 = select(query)
sucursal_2 = sucursal_2[0]

# Estantes de la sucursal 1
query = """ 
    select * from estantes
    where id_sucursal = 1
"""
estantes_1_DB = select(query)

# Estantes de la sucursal 2
query = """ 
    select * from estantes
    where id_sucursal = 2
"""
estantes_2_DB = select(query)

# Stock de la sucursal 1
query = """ 
    select * from stock
    where id_sucursal = 1
"""
stock_1 = select(query)

# Stock de la sucursal 2
query = """ 
    select * from stock
    where id_sucursal = 2
"""
stock_2 = select(query)

#El id de todos nuestros clientes registrados
query = """ select id from cliente
"""
clients = select(query)
clients = clients[0]

#El id de cliente de los clientes en el club
query = """ select id_cliente from club_clientes
"""
club_clients = select(query)
club_clients = club_clients[0]

# Llenar estantes

In [10]:
# LLenar estantes de la sucursal 1
estantes_1 = []

#Dejamos la disponibilidad en cero para que estén llenos
for index, tuple in enumerate(estantes_1_DB):
    estante = Estante(tuple[0], tuple[1], tuple[2], tuple[3])
    estantes_1.append(estante)

for i in range(len(estantes_1)):
    random_product = random.randint(0, len(stock_1)-1)
    llenarEstante(estantes_1[i], stock_1[random_product][1])

    
# LLenar estantes de la sucursal 2
estantes_2 = []
for index, tuple in enumerate(estantes_2_DB):
    estante = Estante(tuple[0], tuple[1], tuple[2], tuple[3])
    estantes_2.append(estante)
    
for i in range(len(estantes_2)):
    random_product = random.randint(0, len(stock_2)-1)
    llenarEstante(estantes_2[i], stock_2[random_product][1])

print(estantes_1[4].disponibilidad)
print(estantes_2[2].disponibilidad)


0
0


# Definir variables principales

In [19]:
#Máximo de personas en las sucursales
max_personas_1 = sucursal_1[2]
max_personas_2 = sucursal_2[2]

#Contador de las personas en las sucursales
cont_personas_1 = 0
cont_personas_2 = 0

#Compradores en los pasillos
compradores_sucursal_1 = []
compradores_sucursal_2 = []

#Compradores en las colas
cola_sucursal_1 = []
cola_sucursal_2 = []

# Ciclo de la simulación

In [27]:
#Fecha de hoy
actualDate = datetime.datetime.now()
while(True):
    
    #Empieza un nuevo día a las 7 am
    print("--------------------- Nuevo Día ---------------------")
    actualDate = actualDate.replace(hour=7,minute=0,second=0)
    print(actualDate)
    
    #Sucursales vacías
    cont_personas_1 = 0
    cont_personas_2 = 0
    
    compradores_sucursal_1 = []
    compradores_sucursal_2 = []
    
    cola_sucursal_1 = []
    cola_sucursal_2 = []
    
    #Carritos de compras vacíos
    carrito_1 = {
                "producto":0,
                "cantidad":0
    }
    carrito_2 = {
                "producto":0,
                "cantidad":0
    }
    
    #cerramos a las 6 de la tarde
    while(actualDate.hour < 18):
        print(actualDate)
        #----Empezamos con la sucursal 1 
        
        #------ Generamos una nueva entrada de un cliente si hay espacio
        if(cont_personas_1 <= max_personas_1):
            cont_personas_1 +=1
            #generamos la identidad del cliente con un número random
            random_id = random.randint(1, 10)
            
            persona = {
                "id":random_id,
                "productos":[],
                "carrito":[]
            }
            compradores_sucursal_1.append(persona)
            print("personas")
            for i in range(len(compradores_sucursal_1)):
                print(compradores_sucursal_1[i]["id"])
            #Revisamos si el cliente es parte del programa de fidelidad
            if(random_id in club_clients):
                #Añadimos una visita a su cuenta si tiene una cuenta
                incrementVisit(1);
            #añadimos la visita a la tabla de visitas
            insertVisit(actualDate, 1);
            
        # El cliente pasa a elegir de los estantes
        #estante random
        #-------- prueba
        for i in range(len(compradores_sucursal_1)):
                #El tamaño de la lista cambia, hay que asegurarse de que no nos salgamos del rango
                if(i < len(compradores_sucursal_1)):
                    #Si el numero random es 1, enviamos el cliente a pagar, sino, a busacar un producto
                    decision = random.randint(1, 3)
                    if(decision == 1):
                        print("---- a pagar ----")
                        print("i",i)
                        print("compradores",len(compradores_sucursal_1))
                        cliente = compradores_sucursal_1.pop(i)
                        cola_sucursal_1.append(cliente)
                    else:
                        random_estante = random.randint(0, len(estantes_1)-1)
                        random_cantidad = random.randint(1, (estantes_1[random_estante].capacidad - estantes_1[random_estante].disponibilidad))
                        #cantidad random de productos
                        #capacidad - disponibilidad = cantidad de productos actuales
        
        
                        #Metemos al carrito los productos
                        compradores_sucursal_1[i]["productos"].append(estantes_1[random_estante].producto)
                        conjunto = {
                            "producto":estantes_1[random_estante].producto,
                            "cantidad":random_cantidad
                        }
                        compradores_sucursal_1[i]["carrito"].append(conjunto)
                    
                        carrito_1["producto"] = estantes_1[random_estante].producto
                        carrito_1["cantidad"] = random_cantidad
                        print("---- llevando productos ----")
                        print(carrito_1["producto"])
                        print(carrito_1["cantidad"])
        
                        #retiramos del estante la cantidad de productos que se llevaron
                        estantes_1[random_estante].disponibilidad += random_cantidad
        
                        #calculamos el límite para hacer refill
                        limite = (estantes_1[random_estante].capacidad*0.2)
        
                        if((estantes_1[random_estante].capacidad - estantes_1[random_estante].disponibilidad) <= limite):
                            print("ALERTA REFILL")
            
                            #Publicamos con el protocolo mqtt
                            enviarAlerta(estantes_1[random_estante].id, 1)
                            llenarEstante(estantes_1[random_estante], estantes_1[random_estante].producto)
            
        
        actualDate += datetime.timedelta(minutes=30)
        time.sleep(0.5)
   
    #Aumentamos la fecha un día
    actualDate += datetime.timedelta(days=1) 

--------------------- Nuevo Día ---------------------
2020-11-09 07:00:00.507053
2020-11-09 07:00:00.507053
personas
10
nueva visita
---- llevando productos ----


TypeError: list indices must be integers or slices, not str

# Funciones auxliares

In [8]:
def insertVisit(date, sucursal):
    theDate = date.timestamp()
    query = f""" INSERT INTO visitas(fecha, id_sucursal)
    VALUES(to_timestamp ( {theDate} ),{sucursal});
    """
    cur = myConnection.cursor()
    cur.execute(query)
    myConnection.commit()
    print("nueva visita")
    
def incrementVisit(id):
    query = f""" 
        UPDATE club_clientes 
           SET visitas = visitas + 1
        WHERE id_cliente = {id};
    """
    cur = myConnection.cursor()
    cur.execute(query)
    myConnection.commit()
    print("Una más al club");
    
def llenarEstante(estante, producto):
    estante.producto = producto
    estante.disponibilidad = 0
    
def enviarAlerta(estante, sucursal):
    payload={
            "mensaje": f"el estante {estante} en la sucursal {sucursal} necesita refill",
            }        
    print (payload)
    client.publish('alertas/estantes',json.dumps(payload),qos=2)